In [ ]:
import warnings
warnings.filterwarnings('ignore')

**All Required Libraries and installations needed**

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

In [ ]:
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from sklearn.metrics import f1_score
!pip install rouge-score
from rouge_score import rouge_scorer
data = pd.read_json("test.json")

 **The quantized Llama2-13b-Chat model**

 https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF

 can use other finetuned models available in the repo too


In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=32)

 **Testing the Model**

In [ ]:
data1 = data_test.iloc[0]
prompt = data1
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer in very brief with simple 2 lines of explanation.

USER: {prompt}

ASSISTANT:
'''

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)
print(response["choices"][0]["text"])

In [ ]:
data_sample = data.head(30)

model_responses = []
for question in data_sample['question']:
    prompt_template = f'''SYSTEM: You are a helpful, respectful and honest assistant. Always give factually coherent answers.

USER: {question}

ASSISTANT:
'''
    response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0, top_p=0.95,
                        repeat_penalty=1.2, top_k=150, echo=False)
    model_responses.append(response["choices"][0]["text"])

data_sample['model_response'] = model_responses

**Applying ROUGE SCORE to evaluate**

In [ ]:
# Apply ROUGE scores
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []
for true_answer_list, model_answer in zip(data_sample['answers'], data_sample['model_response']):
    # Assuming 'true_answer_list' is a list of strings, join them into a single string
    true_answer = ' '.join(true_answer_list)
    scores = rouge_scorer_instance.score(true_answer, model_answer)
    rouge_scores.append(scores)

data_sample['rouge_score'] = rouge_scores

In [ ]:
# Calculate average scores
avg_rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)

print(f'Average ROUGE-1 F1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 F1 Score: {avg_rouge2}')
print(f'Average ROUGE-L F1 Score: {avg_rougeL}')

In [ ]:
# Apply ROUGE scores
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []
for true_answer_list, model_answer in zip(data_sample['context'], data_sample['model_response']):
    # Assuming 'true_answer_list' is a list of strings, join them into a single string
    true_answer = ' '.join(true_answer_list)
    scores = rouge_scorer_instance.score(true_answer, model_answer)
    rouge_scores.append(scores)

data_sample['rouge_score'] = rouge_scores


In [ ]:
# Calculate average scores
avg_rouge1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rouge2 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
avg_rougeL = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)

print(f'Average ROUGE-1 F1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 F1 Score: {avg_rouge2}')
print(f'Average ROUGE-L F1 Score: {avg_rougeL}')

In [ ]:
# Print sample results
for index, row in data_sample.iterrows():
    print(f"Question {index+1}:")
    print(f"Question: {row['question']}")
    print(f"Context: {row['context']}")
    print(f"True Answer: {row['answers']}")
    print(f"Model Response: {row['model_response']}")
    print(f"ROUGE Scores: {row['rouge_score']}")
